In [0]:
%run "../../../config/setup"

In [0]:
%python

%pip install user_agents

In [0]:
%python

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import user_agents

# Define the UDF to parse user_agent and extract device_type
def parse_user_agent(user_agent):
    if user_agent:
        ua = user_agents.parse(user_agent)
        return ua.device.family  # Returns device type (e.g., "iPhone", "Mac", "Spider")
    return None

# Register the UDF
parse_user_agent_udf = udf(parse_user_agent, StringType())

spark.udf.register("parse_user_agent_browser_sql", parse_user_agent_udf)

In [0]:
CREATE OR REPLACE TABLE customers_engagement_summary_gold AS
WITH session_data AS (
  SELECT
    customer_id,
    session_id,
    COUNT(*) AS page_views
  FROM clickstream_customers_silver
  GROUP BY customer_id, session_id
)
SELECT
  c.customer_id,
  COUNT(t.transaction_id) AS total_transactions,
  COUNT(fi.interaction_type) AS total_facebook_actions,
  -- AVG(UNIX_TIMESTAMP(session_end) - UNIX_TIMESTAMP(session_start)) AS avg_session_duration_seconds,
  MODE(cl.page_url) AS most_visited_page,
  MAX(fi.interaction_date) AS last_facebook_interaction,
  MODE(cl.device_type) AS device_type
FROM customers_silver c
LEFT JOIN transactions_silver t ON c.customer_id = t.customer_id
LEFT JOIN facebook_silver fi ON c.customer_id = fi.customer_id
LEFT JOIN session_data s ON c.customer_id = s.customer_id
LEFT JOIN (
  SELECT
    customer_id,
    parse_user_agent_browser_sql(user_agent) AS device_type,
    page_url
  FROM clickstream_customers_silver
) cl ON c.customer_id = cl.customer_id
GROUP BY c.customer_id;